# DATA ANALYSIS

In [ ]:
import pandas as pd
import psycopg2
import psycopg2.extras
from datetime import datetime
import numpy as np
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import warnings
import kats
from kats.utils.decomposition import TimeSeriesDecomposition
from kats.detectors.trend_mk import MKDetector
from kats.consts import TimeSeriesData
from kats.detectors.seasonality import FFTDetector
from kats.detectors.cusum_detection import CUSUMDetector
from kats.detectors.outlier import OutlierDetector
warnings.filterwarnings("ignore")
logging.getLogger().setLevel(logging.ERROR)

## Shared Functions

In [ ]:
def retrieve_data(db_name, username, db_host,  db_password, db_port, query, column):
    conn = psycopg2.connect(dbname=db_name, user=username, host=db_host, password=db_password, port=db_port)
    cur = conn.cursor(column, cursor_factory=psycopg2.extras.DictCursor)    
    cur.execute(query)
    df = cur.fetchall()
    return df                                                    

In [ ]:
#split dataframe based on year
def split_df(df):
    temp1 = []    
    temp2 = []
    for elem in range(0, len(df['data'])):        
        if(df.loc[elem].at["data"].year == 2021):
            temp1.append(df.loc[elem])
        elif(df.loc[elem].at["data"].year == 2022):
            temp2.append(df.loc[elem])
    df1 = pd.DataFrame(temp1).sort_values(by='data') 
    df2 = pd.DataFrame(temp2).sort_values(by='data') 
    return df1, df2

## Global and Weekly Null Values Count 

### Global Null Values

In [ ]:
def count_null_values(db_name, username, db_host, db_password, db_port, query, column, column_name):
    df = retrieve_data(db_name, username, db_host, db_password, db_port, query, column)
    df = pd.DataFrame(np.array(df), columns = ['data', column_name])
    df['data'] = pd.to_datetime(df['data'], format = '%Y-%m-%d %H:%M:%S', utc=True)
    df.set_index('data', inplace = True)  
    print()
    print(column_name)
    print()   
    null_values = df[column_name].isna().sum()
    print('Missing values:', null_values)
    percent_missing_values = null_values * 100 / len(df[column_name])
    print('Percentage of missing values:', percent_missing_values)    
    not_null_values = df[column_name].notna().sum()
    print('Not missing values:', not_null_values)
    nd_values = df[df[column_name] == 'ND'].shape[0]
    print('ND values:', nd_values)
    percent_nd_values = nd_values * 100 / len(df[column_name])
    print('Percentage of ND values:', percent_nd_values)   
    

In [ ]:
#count values for all sensors
atm_list = ['atm05', 'atm06', 'atm07', 'atm10', 'atm11', 'atm12', 'atm13', 'atm14']
columns = ['trs_ppb', 'trs_stato', 'voc_ppm', 'voc_stato', 'c6h6_ppb', 'c6h6_stato', 'h2s_ppb', 'h2s_stato', 'pidvoc_ppb', 'pidvoc_stato']
for atm in atm_list:
    for column in columns:
        count_null_values(db_name, username, host, password, port, 'SELECT data, ' + column + ' FROM ' + atm, 'data', column)

### Weekly Null Values

In [ ]:
def count_null_by_week(db_name, username, db_password, db_port, query,column, column_name):
    df = retrieve_data(db_name, username, db_password, db_port, query, column)        
    df = pd.DataFrame(np.array(df), columns = ['data', 'null_values', 'not_null_values'])
    df['data'] = pd.to_datetime(df['data'], format = '%Y-%m-%d %H:%M:%S', utc=True)
    #print(df['data'].count())
    df['percentage_null_values'] = ((df['null_values'] / (df['null_values'] + df['not_null_values']))*100)    
    for value in df['percentage_null_values']: 
        df['percentage_null_values'] = df['percentage_null_values'].replace(value, round(value, 2))    
    df1, df2 = split_df(df)
    return df1, df2

In [ ]:
#show or save weekly null values 
def show_and_save_weekly_nulls(df, save_path, atm, column, year):  
    print()
    print('Chemical: ', column)        
    print('Values less than 10% - 2021: ', len(df[df['percentage_null_values'] < 10]))
    print()
    print('Values less than 10% - 2022: ', len(df[df['percentage_null_values'] < 10]))
    print()
    save_path = '../Notebook/Results/Null_Values' + atm.upper()
    fig, ax = plt.subplots()
    ax.axis('off')
    ax.axis('tight')
    t = ax.table(cellText = df.values, colWidths = [0.9]*len(df.columns),  colLabels=df.columns,  loc='center', cellLoc = 'center')
    t.auto_set_font_size(False) 
    t.set_fontsize(10) 
    plt.show()
    plt.savefig(save_path + year + column + '.jpg', bbox_inches='tight')    


### Weekly ND Values

In [ ]:
def count_nd_by_week(db_name, username, db_host , db_password,db_port, query_nd, query_not_nd, column, column_name):
    df = retrieve_data(db_name, username, db_host, db_password, db_port, query_nd, column)        
    df = pd.DataFrame(np.array(df), columns = ['data', 'nd_values'])
    df['data'] = pd.to_datetime(df['data'], format = '%Y-%m-%d %H:%M:%S', utc=True)    
    df_temp = retrieve_data(db_name, username, db_host, db_password, db_port, query_not_nd, column)
    df_temp = pd.DataFrame(np.array(df_temp), columns = ['data', 'not_nd_values'])
    df = df.join(df_temp['not_nd_values'])        
    df['percentage_nd_values'] = ((df['nd_values'] / (df['nd_values'] + df['not_nd_values']))*100)    
    for value in df['percentage_nd_values']: 
        df['percentage_nd_values'] = df['percentage_nd_values'].replace(value, round(value, 2))
    df1, df2 = split_df(df)
    return df1, df2

In [ ]:
def show_and_save_weekly_nd(df, save_path, atm, column, year):     
    print()
    print('Chemical': column)
    print(len(df['percentage_nd_values']))
    print('Values less than 10% - '+ year + ': ', len(df[df['percentage_nd_values'] < 10]))
    print()
    fig, ax = plt.subplots()
    ax.axis('off')
    ax.axis('tight')
    t = ax.table(cellText = df.values, colWidths = [0.9]*len(df.columns),  colLabels=df.columns,  loc='center', cellLoc = 'center')
    t.auto_set_font_size(False) 
    t.set_fontsize(10)
    #ax.set_xlabel('Tabella valori nulli ' + column_name)
    #fig.tight_layout()    
    plt.show()
    plt.savefig(save_path + year + atm + column + '.jpg', bbox_inches='tight')

### Nulls and ND function execution

In [ ]:
atm_list = ['atm05', 'atm06', 'atm07', 'atm10', 'atm11', 'atm12', 'atm13', 'atm14']
columns = ['trs_ppb', 'trs_stato', 'voc_ppm', 'voc_stato', 'c6h6_ppb', 'c6h6_stato', 'h2s_ppb', 'h2s_stato', 'pidvoc_ppb', 'pidvoc_stato']
for atm in atm_list:
    save_path_nulls = '../Notebook/Results/Nulls_ND_Values/Nulls_Weekly/nulls_tables' + atm.upper()
    save_path_nd = '../Notebook/Results/Nulls_ND_Values/Nd_Weekly/nd_tables' + atm.upper()
    for column in columns:
        #null values
        query_nulls = "select time_bucket('1 week', data) as bucket, sum(case when " + column + " is null then 1 else 0 end) null_values,count(" + column + ") not_nulls from " + atm + " GROUP BY bucket"                    
        df1_nulls, df2_nulls = count_null_by_week(db_name, username, host, password, port, query, 'data', column)            
        show_and_save_weekly_nulls(df1_nulls, save_path_nulls, atm, column, '2021')
        show_and_save_weekly_nulls(df2_nulls, save_path_nulls, atm, column, '2022')
        #nd values
        query_nd = "select time_bucket('1 week', data) as bucket, count(*) as nd_values from " + atm + " WHERE " +  column + " = 'ND' GROUP BY bucket "              
        query_not_nd = "select time_bucket('1 week', data) as bucket, count(*) as not_nd_values from " + atm + " WHERE " +  column + " <> 'ND' GROUP BY bucket "      
        df1_nd, df2_nd = count_nd_by_week(db_name, username, host, password, port, query_nd, query_not_nd 'data', column)
        show_and_save_weekly_nd(df1_nd, save_path_nd, atm, column, '2021')
        show_and_save_weekly_nd(df2_nd, save_path_nd, atm, column, '2022')

### Global charts results showed after getting percentage results

In [ ]:
#Charts of percentage of null values by chemical detection
labels = ['ATM05', 'ATM06', 'ATM07', 'ATM10', 'ATM11', 'ATM12', 'ATM13', 'ATM14']
values = [9, 11.4, 5.6, 6.7, 6.7, 8.6, 12.7, 7.4]
plt.bar(labels, values)
plt.ylabel("Percentage")
plt.title("Percentage of null values by chemical detection")
plt.show()

In [ ]:
#Charts of percentage of null values by detection status
labels = ['ATM05', 'ATM06', 'ATM07', 'ATM10', 'ATM11', 'ATM12', 'ATM13', 'ATM14']
null_values = [0.67, 4.6, 0.85, 0.68, 0.68, 0.62, 1.12, 0.68]
nd_values = [8.3, 6.8, 4.7, 6, 6, 8, 11.6, 6.7]
x = np.arange(len(labels))
width = 0.4
fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, null_values, width, label='Valori nulli')
rects2 = ax.bar(x + width/2, nd_values, width, label='Valori ND')
ax.set_ylabel('Percentage')
ax.set_title('Percentage null and ND values by detection status')
ax.set_xticks(x, labels)
ax.legend()
ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)
fig.tight_layout()
plt.show()

## Values above and below maximum and minimum sensors thresholds

### Global Count

In [ ]:
#Counting lower or higher values than detectable by the sensor - Global
def count_vs_treshold(db_name, username, db_host, db_password, db_port, query, cur_column, column_name, min_value, max_value):
    df = retrieve_data(db_name, username, db_host, db_password, db_port, query, cur_column)    
    df = pd.DataFrame(np.array(df), columns = ['data', column_name])
    df['data'] = pd.to_datetime(df['data'], format = '%Y-%m-%d %H:%M:%S', utc=True)
    df.set_index('data', inplace = True)
    df[column_name] = df[column_name].astype(float)    
    count_min_values = len(df[df[column_name] < min_value])
    percent_min_values = count_min_values * 100 / len(df[column_name])
    count_max_values = len(df[df[column_name] > max_value])
    percent_max_values = count_max_values * 100 / len(df[column_name])
    print()
    print(column_name)
    print('Values below minimum threshold: ', count_min_values)
    print('Percentage values below minimum threshold: ', percent_min_values)
    print('Values above maximum threshold', count_max_values)
    print('Percentage values above maximum threshold: ', percent_max_values)

In [ ]:
#Charts of percentage of values below minimun threshold (there aren't values above thresholds)
def plot_below_thresholds(labels, values, name):
    fig, ax = plt.subplots()        
    bars = ax.bar(labels, values)
    ax.bar_label(bars)
    plt.ylabel("Percentage %")
    plt.title("Percentage of values below minimun threshold " + name)
    plt.show()

In [ ]:
labels = ['trs_ppb', 'voc_ppm', 'c6h6_ppb', 'h2s_ppb', 'pidvoc_ppb']
plot_below_thresholds(labels, [0, 90, 51.87, 38.21, 11.73], 'ATM05', )
plot_below_thresholds(labels, [0, 88.6, 42.18, 25.77, 8.5], 'ATM06')
plot_below_thresholds(labels, [0, 93.54, 53, 0.002, 32.32], 'ATM07')
plot_below_thresholds(labels, [0, 82, 26.93, 0, 21.32], 'ATM10')
plot_below_thresholds(labels, [0, 91.36, 57.55, 24.33, 8.2], 'ATM11')
plot_below_thresholds(labels, [0, 90, 22.54, 21, 19.56], 'ATM12')
plot_below_thresholds(labels, [0, 86.38, 19.29, 27, 29.42], 'ATM13')
plot_below_thresholds(labels, [0, 87, 31.29, 0, 9.14], 'ATM14')


In [ ]:
labels = ['ATM05', 'AMT06', 'ATM07', 'ATM10', 'ATM11', 'ATM12', 'ATM13', 'ATM14']
plot_below_thresholds(labels, [90, 88.6, 93.54, 82, 91.36, 90, 86.38, 87], 'voc_ppm')
plot_below_thresholds(labels, [51.87, 42.18, 53, 26.93, 57.55, 22.54, 19.29, 31.29], 'c6h6_ppb')
plot_below_thresholds(labels, [38.21, 25.77, 0.002, 0,  24.33, 21, 27, 0], 'h2s_ppb')
plot_below_thresholds(labels, [11.74, 8.5, 32.32, 21.32, 8.2, 19.56, 29.42, 9.14], 'pidvoc_ppb')

### Weekly Count

In [ ]:
def count_weekly_vs_threshold(db_name, username, db_host, db_password, db_port, column, column_name, min_value, max_value, atm):
    query = "select time_bucket('1 week', data) as bucket, COUNT(*) AS total_values, COUNT(CASE WHEN " + column_name + " < " + str(min_value) + " THEN 1 END) AS min_values, COUNT(CASE WHEN " + column_name + " > " + str(max_value) + " THEN 1 END) AS max_values from " + atm + " GROUP BY bucket"
    df = retrieve_data(db_name, username, db_host, db_password, db_port, query, column)        
    df = pd.DataFrame(np.array(df), columns = ['data', 'total_values', 'min_values', 'max_values',])                         
    df['data'] = pd.to_datetime(df['data'], format = '%Y-%m-%d %H:%M:%S', utc=True)                            
    df['min_percentage'] = ((df['min_values'] / (df['total_values'])*100))    
    df['max_percentage'] = ((df['max_values'] / (df['total_values'])*100))    
    for value in df['min_percentage']: 
        df['min_percentage'] = df['min_percentage'].replace(value, round(value, 2))
    for value in df['max_percentage']: 
        df['max_percentage'] = df['max_percentage'].replace(value, round(value, 2))
    df1, df2 = split_df(df)
    return df1, df2

In [ ]:
def weekly_below_charts(df, save_path, year, atm, column):
    fig, ax = plt.subplots()        
    fig.set_size_inches(18.5, 10.5)
    bars = ax.bar(df['data'], df['min_percentage'])
    ax.bar_label(bars)
    ax.set_xlabel('Week')
    ax.set_ylabel('Percentage')
    ax.set_title('Weekly percentage values below the threshold' + column)
    fig.tight_layout()
    plt.savefig(save_path + year + atm + column + '.jpg', bbox_inches='tight')
    

In [ ]:
def weekly_below_table(df, save_path, year, atm, column):
    fig, ax = plt.subplots()
    ax.axis('off')
    ax.axis('tight')
    t = ax.table(cellText = df.values, colWidths = [0.9]*len(df.columns),  colLabels=df.columns,  loc='center', cellLoc = 'center')
    t.auto_set_font_size(False) 
    t.set_fontsize(10)
    plt.savefig(save_path + year + atm + column + '.jpg', bbox_inches='tight')

In [ ]:
columns = ['trs_ppb', 'voc_ppm', 'c6h6_ppb', 'h2s_ppb', 'pidvoc_ppb']
atm_names = ['atm05', 'atm06', 'atm07', 'atm10', 'atm11', 'atm12', 'atm13', 'atm14']
dic_min_threshold = {"trs_ppb" : 0, "voc_ppm" : 0.6, "c6h6_ppb" : 0.1, "h2s_ppb" : 2, "pidvoc_ppb" : 0}
dic_max_threshold = {"trs_ppb" : 10000, "voc_ppm" : 25, "c6h6_ppb" : 30, "h2s_ppb" : 3000, "pidvoc_ppb" : 40000}

for atm in atm_names:
    path_weekly_charts = '../Notebook/Results/Values_VS_Thresholds/chart' + atm.upper()
    path_weekly_table = '../Notebook/Results/Values_VS_Thresholds/chart' + atm.upper()
    for column in columns:
        #global count
        count_vs_treshold(db_name, username, host, password, port, 'SELECT data, ' + column + ' FROM ' + atm, 'data', column, dic_min_threshold[column], dic_max_threshold[column])
        #weekly count
        df1, df2 = count_weekly_vs_threshold(db_name, username, host, password, port, 'data', column, dic_min_threshold[column], dic_max_threshold[column], atm)            
        weekly_below_charts(df1, path_weekly_charts, '2021', atm, column)
        weekly_below_charts(df2, path_weekly_charts, '2022', atm, column)
        weekly_below_table(df1, path_weekly_table, '2021', atm, column)
        weekly_below_table(df2, path_weekly_table, '2022', atm, column)

## Weekly Average

In [ ]:
def weekly_average(db_name, username, db_host, db_password, db_port, query, cur_column, column_name):
    df = retrieve_data(db_name, username, db_host, db_password, db_port, query, cur_column)    
    df = pd.DataFrame(np.array(df), columns = ['data', 'weekly_average'])
    df['data'] = pd.to_datetime(df['data'], format = '%Y-%m-%d %H:%M:%S', utc=True)
    #df.set_index('data', inplace = True)
    #df[column_name] = df[column_name].astype(float)
    df.sort_index(inplace = True)     
    df1, df2 = split_df(df)
    return df1, df2

In [ ]:
def show_and_save_weekly_average(df, save_path, year):    
    fig, ax = plt.subplots()  
    plt.plot(df['data'], df['weekly_average'])
    fig.set_size_inches(18.5, 10.5)
    ax.set_xlabel('Week')
    ax.set_ylabel('Average')
    ax.set_title('Weekly average ' + column)
    fig.tight_layout()
    plt.show()
    plt.savefig(save_path + year + atm + column + '.jpg'+ column + '.jpg', bbox_inches='tight')

In [ ]:
atm_list = ['atm05', 'atm06', 'atm07', 'atm10', 'atm11', 'atm12', 'atm13', 'atm14']
columns = ['trs_ppb', 'voc_ppm', 'c6h6_ppb','h2s_ppb', 'pidvoc_ppb']
for atm in atm_list:    
    for column in columns:
        save_path = '../Notebook/Results/Weekly_Average/weekly_average' + atm.upper()
        query = "select time_bucket('1 week', data) as bucket, avg( " + column + ") FROM " + atm + " GROUP BY bucket"
        df1, df2 = weekly_average(db_name, username, host, password, port, query, 'data', column)
        show_and_save_weekly_average(df1, save_path, '2021')
        show_and_save_weekly_average(df2, save_path, '2022')
    

## Outliers Analysis
 - Outliers
 - Outliers compared to Weekly Average

In [ ]:
def outliers(df):
    df_temp = df.rename(columns={"data":"time"})    
    df_temp = TimeSeriesData(df_temp)            
    outlier_detector = OutlierDetector(df_temp, "additive")
    outlier_detector.detector()
    outliers = outlier_detector.outliers              
    df_out1 = []            
    for data in df.index:                 
        if df1.loc[data].at['data'] in outliers[0]:                       
            df_out.append(df.loc[data])  
                    
    if len(df_out) != 0:
        df_out = pd.DataFrame(np.array(df_out), columns = ['data', column])
    return df_out            
        

In [ ]:
def outliers_detection (column, atm):
    query = 'SELECT data, ' + column + ' FROM ' + atm            
    df = retrieve_data(db_name, username, db_host, db_password, db_port, query, cur_column)    
    df pd.DataFrame(np.array(df), columns = ['data', column_name])
    df['data'] = pd.to_datetime(df['data'], format = '%Y-%m-%d %H:%M:%S', utc=True)    
    df[column_name] = df[column_name].astype(float)
    df.interpolate(method ='ffill', limit_direction='forward', inplace=True)
    df1, df2 = split_df(df) 
    df1_out = outliers(df1)
    df2_out = outliers(df2)
    return df1_out, df2_out

In [ ]:
def save_outliers_to_file(df, save_path, file_name):
    with open(save_path + file_name, "a") as f:
        print(atm + ' ' + column, file=f)
        print( file=f)                                
        outlier_detector = OutlierDetector(df, "additive")
        outlier_detector.detector()
        outliers = outlier_detector.outliers
        print(outliers[0], file=f)
        print( file=f)

In [ ]:
def outliers_chart(df):
    fig, ax = plt.subplots()  
    #fig.set_size_inches(18, 10)                 
    ax.scatter(df_out['data'], df_out[column], alpha=1.0)
    ax.set_title('Outliers ' + column)
    ax.set_xlabel('Week')
    ax.set_ylabel('Outlier')
    plt.xticks(rotation=45) 
    #plt.show()
    #plt.savefig(save_path + atm.upper() + ' outliers ' + column + '2021' + '.jpg', bbox_inches='tight')

In [ ]:
def outliers_chart_vs_average(df, df_average):
    fig, ax = plt.subplots(2)  
    fig.set_size_inches(18, 10)                 
    ax[0].plot(df_average['data'], df_average['weekly_average'])   
    ax[0].set_title('Weekly average ' + column)
    ax[0].set_xlabel('Week')
    ax[0].set_ylabel('Average')
    ax[1].scatter(df_out['data'], df_out[column], alpha=1.0)
    ax[1].set_title('Outliers ' + column)
    ax[1].set_xlabel('Week')
    ax[1].set_ylabel('Outlier')
    plt.xticks(rotation=45) 
    #plt.show()
    #plt.savefig(save_path + atm.upper() + ' outliers ' + column + '2021' + '.jpg', bbox_inches='tight')

In [ ]:
#outliers charts
columns = ['trs_ppb', 'voc_ppm', 'c6h6_ppb', 'h2s_ppb', 'pidvoc_ppb']
atm_names = ['atm05', 'atm06', 'atm07', 'atm10', 'atm11', 'atm12', 'atm13', 'atm14']
for atm in atm_names:
        for column in columns:
            df1_out, df2_out = outliers_detection (column, atm)
            outliers_chart(df1_out)
            outliers_chart(df2_out)

In [ ]:
#outliers compared to weekly average charts
columns = ['trs_ppb', 'voc_ppm', 'c6h6_ppb', 'h2s_ppb', 'pidvoc_ppb']
atm_names = ['atm05', 'atm06', 'atm07', 'atm10', 'atm11', 'atm12', 'atm13', 'atm14']
for atm in atm_names:
    for column in columns:
        query_average = "select time_bucket('1 week', data) as bucket, avg( " + column + ") FROM " + atm + " GROUP BY bucket"
        df1_average, df2_average = weekly_average(db_name, username, host, password, port, query_average, 'data', column)
        df1_out, df2_out = outliers_detection (save_path, file_name1, file_name2)
        outliers_chart_vs_average(df1_out, df1_average)
        outliers_chart_vs_average(df2_out, df2_average)

In [ ]:
#save outliers to .txt
columns = ['trs_ppb', 'voc_ppm', 'c6h6_ppb', 'h2s_ppb', 'pidvoc_ppb']
atm_names = ['atm05', 'atm06', 'atm07', 'atm10', 'atm11', 'atm12', 'atm13', 'atm14']
for atm in atm_names:
        for column in columns:
            df1_out, df2_out = outliers_detection (column, atm)
            save_outliers_to_file(df1_out, '../Notebook/Results/', 'Outliers 2021')
            save_outliers_to_file(df2_out, '../Notebook/Results/', 'Ouliers 2022')

## Seasonality Analysis
 - Seasonality with statsmodel
 - seasonality with kats
 - change detection kats

### Statsmodel

In [ ]:
#save seasonality charts as jpg 
def seasonality_statsmodel_plot(df, column_name, atm, year):    
    y = df[column_name]
    #y.plot(title=column_name);
    seasonal_decomp = seasonal_decompose(y, model="additive", period=56)
    fig, axes = plt.subplots(4, 1, sharex=True)
    fig.set_size_inches(25.5, 20.5)
    seasonal_decomp.observed.plot(ax=axes[0], legend=False, color='r')
    axes[0].set_ylabel('Observed')
    seasonal_decomp.trend.plot(ax=axes[1], legend=False, color='g')
    axes[1].set_ylabel('Trend')
    seasonal_decomp.seasonal.plot(ax=axes[2], legend=False)
    axes[2].set_ylabel('Seasonal')
    seasonal_decomp.resid.plot(ax=axes[3], legend=False, color='k')
    axes[3].set_ylabel('Residual')
    plt.savefig(save_path + atm.upper() + 'seasonality ' + column + year + '.jpg', bbox_inches='tight')
    

In [ ]:
def seasonality_analisys_statsmodel(save_path, db_name, username, db_host, db_password, db_port, query, cur_column, column_name):    
    df = retrieve_data(db_name, username, db_host, db_password, db_port, query, cur_column)    
    df = pd.DataFrame(np.array(df), columns = ['data', column_name])
    df['data'] = pd.to_datetime(df['data'], format = '%Y-%m-%d %H:%M:%S', utc=True)
    #df.set_index('data', inplace = True)
    df[column_name] = df[column_name].astype(float)
    '''
    print('Summary')
    print('')
    print(df.describe())
    print()
    print('Null values')
    print()
    print(df.isnull().sum())
    '''
    df.interpolate(method ='ffill', limit_direction='forward', inplace=True)
    df1, df2 = split_df(df)
    df1.set_index('data', inplace = True)
    df2.set_index('data', inplace = True)
    
    seasonality_statsmodel_plot(df1, column_name, atm, '2021')
    seasonality_statsmodel_plot(df2, column_name, atm, '2022')

In [ ]:
columns = ['trs_ppb', 'voc_ppm', 'c6h6_ppb', 'h2s_ppb', 'pidvoc_ppb']
atm_names = ['atm05', 'atm06', 'atm07', 'atm10', 'atm11', 'atm12', 'atm13', 'atm14']
for atm in atm_names:
    for column in columns:
        query = 'SELECT data, ' + column + ' FROM ' + atm        
        seasonality_analisys_statsmodel('../Analysis/seasonality_statsmodel/', db_name, username, host, password, port, query, 'data', column)

### Kats

In [ ]:
def seasonality_decomposition(df, year):
    df = df.rename(columns={"data":"time"})    
    df = TimeSeriesData(df)
    decompose = TimeSeriesDecomposition(df1, decomposition='additive', method='STL')
    _ = decompose.decomposer()
    decompose.plot()
    plt.xticks(rotation=45)
    plt.savefig(save_path + atm.upper() + 'seasonality ' + column + year + '.jpg', bbox_inches='tight')

In [ ]:
#seasonality analysis with kats
def seasonality_kats(db_name, username, db_host, db_password, db_port, query, cur_column, column_name, save_path):    
    df = retrieve_data(db_name, username, db_host, db_password, db_port, query, cur_column)    
    df = pd.DataFrame(np.array(df), columns = ['data', column_name])
    df['data'] = pd.to_datetime(df['data'], format = '%Y-%m-%d %H:%M:%S', utc=True)
    #df.set_index('data', inplace = True)
    df[column_name] = df[column_name].astype(float)
    df.interpolate(method ='ffill', limit_direction='forward', inplace=True)
    df1, df2 = split_df(df)
    seasonality_decomposition(df1, '2021')
    seasonality_decomposition(df2, '2022')

In [ ]:
columns = ['trs_ppb', 'voc_ppm', 'c6h6_ppb', 'h2s_ppb', 'pidvoc_ppb']
atm_names = ['atm05', 'atm06', 'atm07', 'atm10', 'atm11', 'atm12', 'atm13', 'atm14']
for atm in atm_names:
    for column in columns:
        query = 'SELECT data, ' + column + ' FROM ' + atm
        seasonality_kats(db_name, username, host, password, port, query, 'data', column, 'Notebook/Results/')     

In [ ]:
def save_seasonality_to_file(df, save_path, file_name, atm, column):
    with open(save_path + file_name, "a") as f:
        print(atm + ' ' + column, file=f)
        print( file=f) 
        fft_detector = FFTDetector(df)
        print(fft_detector.detector(), file=f)
        print( file=f)

In [ ]:
#save seasonality info to .txt
columns = ['trs_ppb', 'voc_ppm', 'c6h6_ppb', 'h2s_ppb', 'pidvoc_ppb']
atm_names = ['atm05', 'atm06', 'atm07', 'atm10', 'atm11', 'atm12', 'atm13', 'atm14']
for atm in atm_names:
    for column in columns:
        query = 'SELECT data, ' + column + ' FROM ' + atm
        df1, df2 = seasonality_kats(db_name, username, host, password, port, query, 'data', column)     
        save_seasonality_to_file(df1, '../Notebook/Results', 'seasonality2021.txt', atm, column)
        save_seasonality_to_file(df2, '../Notebook/Results', 'seasonality2021.txt', atm, column):

In [ ]:
def change_detection(df, save_path, year):
    detector = CUSUMDetector(df)
    change_points = detector.detector(change_directions=["increase", "decrease"])        
    detector.plot(change_points1)
    plt.rcParams["figure.figsize"] = (25,18)
    plt.xticks(rotation=45)
    #plt.savefig(save_path + atm.upper() + 'change_point ' + column + year + '.jpg', bbox_inches='tight')

In [ ]:
#display change detection point charts or save them on disk (comment/uncomment the right line)
    columns = ['trs_ppb', 'voc_ppm', 'c6h6_ppb', 'h2s_ppb', 'pidvoc_ppb']
    atm_names = ['atm05', 'atm06', 'atm07', 'atm10', 'atm11', 'atm12', 'atm13', 'atm14']
    for atm in atm_names:
        for column in columns:            
            query = 'SELECT data, ' + column + ' FROM ' + atm
            df = retrieve_data(db_name, username, host, password, port, query, 'data', column)
            df1, df2 = split_df(df)
            change_detection(df1, '../Notebook/Results', '2021')
            change_detection(df2, '../Notebook/Results', '2022')